# Filtrado Colaborativo: *Probabilistic Matrix Factorization*

Durante los inicios del filtrado colaborativo el algoritmo de KNN era el más empleado debido a los buenos resultados que reportaba y a la facilidad con la que podían explicarse sus recomendaciones. Sin embargo, este algoritmo tiene una gran desventaja: su escalabilidad. El algoritmo de KNN funciona bien para datasets de tamaño medio, pero, a medida que el dataset crece, los tiempos de cómputo para obtener las recomendaciones se vuelven inasumibles. Aumentar el número de usuarios y/o el número de items implica ralentizar el cálculo de las similaridades, la búsqueda de los k vecinos y el número de predicciones a realizar.

Como consecuencia de estos problemas y del gran empuje que supuso el [Netflix Prize](https://www.netflixprize.com/) (concurso que ofrecía una recompensa de 1M de dólares al equipo que consiguiera mejorar el RMSE en el dataset de Netflix) comenzaron a ganar fuerza los sistemas de filtrado colaborativo basados en modelos, más concretamente los basados en modelos de factorización matricial.

El **filtrado colaborativo basado en factorización matricial** se basa en la siguiente idea: las votaciones que los usuarios realizan a los items están condicionadas por una serie de factores latentes intrínsecos a los usuarios y los items. Ilustremos esto con un ejemplo. Supongamos un sistema de recomendación de películas. Lo que postula la factorización matricial es que los usuarios votan las películas basándose no sólo en la propia película, sino que lo hacen basándose en las características que describen esa película. Si a un usuario le gustan las películas de acción con un toque de comedia, es muy probable que le gusten todas las películas de acción con un toque de comedia. Los algoritmos de filtrado colaborativo buscan estas propiedades intrínsecas al dominio en el que se realizan las recomendaciones y las denominan **factores latentes** u ocultos. Es importante resaltar que estos factores son ocultos, y aunque en el ejemplo de la recomendación de películas podamos suponer que se trata de géneros de cine, el modelo nunca nos va a indicar con qué género se corresponde cada factor.

Matemáticamente, la factorización matricial consiste en encontrar las matrices $P$ y $Q$ que satisfagan la siguiente expresión:

$$R \approx P \cdot Q$$

En esta expresión:

- $R$ representa la matriz (dispersa) con las votaciones de los usuarios (filas) a los items (columnas).
- $P$ representa las matriz (densa) de factores de los usuarios (filas) con los *k* factores latentes (columnas).
- $Q$ representa las matriz (densa) de factores de los items (columnas) con los *k* factores latentes (filas).

Como vemos, los modelos tienen un parámetro que será necesario tunear con el fin de ajustar el modelo a cada dataset. Este parámetro ***k*** representa el número de factores latentes de nuestro modelo.

Desarrollando la expresión anterior, podemos inferir que la predicción de voto de un usuario $u$ a un item $i$ queda como:

$$\hat{r}_{u,i} = \vec{p}_u \cdot \vec{q}_i$$

Dónde $\vec{p}_u$ representa un vector fila de la matriz $P$ con los factores latentes del usuario $u$ y $\vec{q}_i$ representa un vector columna de la matriz $Q$ con los factores latentes del item $i$.

Por lo tanto, podemos plantear la búsqueda de los factores latentes como un problema de optimización, en el cual buscamos minimizar el error cometido en los votos conocidos:

$$\min_{p,q} \sum_{(u,i) \in R} ( r_{u,i} - \vec{p}_u \cdot \vec{q}_i)^2$$

Expresión a la que podemos añadir una regularización para evitar el *overfitting*:

$$\min_{p,q} \sum_{(u,i) \in R} ( r_{u,i} - \vec{p}_u \cdot \vec{q}_i)^2 + \lambda (||\vec{p}_u||^2 + ||\vec{q}_i||^2)$$

Es posible resolver este problema mediante la técnica de descenso de gradiente, para lo cual debemos encontrar la derivada de la expresión anterior respecto del $\vec{p}_u$ y $\vec{q}_i$.

Una vez entrenado el modelo, las matrices $P$ y $Q$ son aprendidas y no necesitan modificarse hasta que la matriz de votaciones cambie sustancialmente. Obtener una predicción una vez el modelo ha aprendido implica, simplemente, realizar el producto escalar de dos vectores de dimensión *k*, que, por lo general, suele ser un valor pequeño.

A este algoritmo se le conoce como ***Probabilistic Matriz Factorization (PMF)***.


## Carga del dataset

Para ilustar mejor el funcionamiento del algoritmo PMF, vamos a desarrollar una implementación del mismo.

Para ello usaremos el dataset de [MovieLens 100K](https://grouplens.org/datasets/movielens/) que contiene 100.000 votos de 943 usuarios sobre 1682 películas. Este dataset ha sido dividido en votaciones de entrenamiento (80%) y votaciones de test (20%). Además, los códigos de usuarios e items, han sido modificados para que comience en 0 y terminen en el número de (usuarios / items) - 1.

Inicialmente definimos algunas constantes que nos serán necesarias durante la codificación del algoritmo:

In [2]:
import urllib.request
import random
import numpy as np
from tqdm import tqdm

In [3]:
NUM_USERS = 943
NUM_ITEMS = 1682

MIN_RATING = 1
MAX_RATING = 5

Y cargamos la matriz con las votaciones de entrenamiento:

In [4]:
ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

training_file = urllib.request.urlopen("https://drive.upm.es/s/tDdluElfGInyUnU/download")
for line in training_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  ratings[int(u)][int(i)] = int(rating)

In [5]:
ratings_np = np.array(ratings)
ratings_np[ratings_np == None] = np.nan
ratings_np = ratings_np.astype(np.float64)

Del mismo modo, cargamos la matriz de votaciones de test:

In [6]:
test_ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

test_file = urllib.request.urlopen("https://drive.upm.es/s/Jn75Vg6okOPsgZu/download")
for line in test_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  test_ratings[int(u)][int(i)] = int(rating)

In [7]:
test_ratings_np = np.array(test_ratings)
test_ratings_np[test_ratings_np == None] = np.nan
test_ratings_np = test_ratings_np.astype(np.float64)

## Inicialización del modelo

Definimos los parámetros necesarios para implementar la factorización matricial mediante PMF.

In [8]:
NUM_FACTORS = 7
LEARNING_RATE = 0.001 # gamma
REGULARIZATION = 0.1 # lambda

Inicializamos las matrices de factores con valores uniformes aleatorios en el intervalo \[0, 1].

In [9]:
p = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)]
q = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)]

In [10]:
p_np = np.array(p) #np.random.random((NUM_USERS, NUM_FACTORS))
q_np = np.array(q) #np.random.random((NUM_ITEMS, NUM_FACTORS))

## Cálculo de las predicciones

Como hemos comentado, calcular la predicción del voto del usuario *u* al item *i* implicar realizar el producto escalar de sus vectores de factores. La siguiente función realiza esta operación:


In [12]:
def compute_prediction_old(p_u, q_i):
    # prediction = 0
    # for k in range(NUM_FACTORS):
    #     prediction += p_u[k] * q_i[k]
    return sum(p_u[k] * q_i[k] for k in range(NUM_FACTORS))

In [17]:
compute_prediction_old(p[0], q[0])

2.3245684876076944

In [14]:
def compute_prediction(p_u, q_i):
    return np.dot(p_u, q_i)

In [15]:
# Al ser pocos factores vamos a tener que tener cuidado al vectorizar porque 
# podemos acabar haciéndolo más lento que con python nativo
%timeit -n 100_000 compute_prediction_old(p[0], q[0])
%timeit -n 100_000 compute_prediction_old(p[0], q[0])
%timeit -n 100_000 compute_prediction(p_np[0], q_np[0])

915 ns ± 81.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
924 ns ± 19 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
838 ns ± 29 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
compute_prediction(p_np[0], q_np[0])

2.3424906359092708

## Aprendizaje de los factores latentes

El proceso de entrenamiento implicar aplicar las operaciones de actualización de las matrices de factores hasta que el algoritmo converja. En general, esta convergencia suele prefijarse como el número de iteraciones que realizamos sobre las operaciones de actualización:

In [17]:
NUM_ITERATIONS = 10

Es importante resaltar que sólo debemos actualizar las matrices $P$ y $Q$ empleando los votos existentes en la matriz $R$.

El siguiente código ejemplifica el proceso de entrenamiento del algoritmo:

In [18]:
for it in tqdm(range(NUM_ITERATIONS)):
    print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))

    updated_p = list(p) # clone p matrix
    updated_q = list(q) # clone q matrix

    for u in range(NUM_USERS):
        for i in range(NUM_ITEMS):
            if ratings[u][i] != None:

                prediction = compute_prediction_old(p[u], q[i])
                rating = ratings[u][i]
                error = rating - prediction

                for k in range(NUM_FACTORS):
                    updated_p[u][k] += LEARNING_RATE * (error * q[i][k] - REGULARIZATION * p[u][k])
                    updated_q[i][k] += LEARNING_RATE * (error * p[u][k] - REGULARIZATION * q[i][k])

    p = updated_p
    q = updated_q

  0%|          | 0/10 [00:00<?, ?it/s]

Iteración 1 de 10


 10%|█         | 1/10 [00:00<00:04,  1.99it/s]

Iteración 2 de 10


 20%|██        | 2/10 [00:01<00:04,  2.00it/s]

...

 80%|████████  | 8/10 [00:04<00:01,  1.90it/s]

Iteración 9 de 10


 90%|█████████ | 9/10 [00:04<00:00,  1.86it/s]

Iteración 10 de 10


100%|██████████| 10/10 [00:05<00:00,  1.91it/s]


In [21]:
import pandas as pd

# Convert ratings to a pandas DataFrame with MultiIndex for sparse representation
ratings_data = []
for u in range(NUM_USERS):
    for i in range(NUM_ITEMS):
        if ratings[u][i] is not None:
            ratings_data.append((u, i, ratings[u][i]))

In [22]:
ratings_df = pd.DataFrame(ratings_data, columns=['user', 'item', 'rating'])
ratings_df.set_index(['user', 'item'], inplace=True)

for it in tqdm(range(NUM_ITERATIONS)):
    print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))

    updated_p = list(p)  # clone p matrix
    updated_q = list(q)  # clone q matrix

    # Iterate through ratings DataFrame instead of nested loops
    for (u, i), row in ratings_df.iterrows():
        prediction = compute_prediction_old(p[u], q[i])
        rating = row['rating']
        error = rating - prediction

        for k in range(NUM_FACTORS):
            updated_p[u][k] += LEARNING_RATE * (error * q[i][k] - REGULARIZATION * p[u][k])
            updated_q[i][k] += LEARNING_RATE * (error * p[u][k] - REGULARIZATION * q[i][k])

    p = updated_p
    q = updated_q

  0%|          | 0/10 [00:00<?, ?it/s]

Iteración 1 de 10


 10%|█         | 1/10 [00:02<00:20,  2.23s/it]

Iteración 2 de 10


 20%|██        | 2/10 [00:04<00:16,  2.12s/it]

...

 80%|████████  | 8/10 [00:17<00:04,  2.18s/it]

Iteración 9 de 10


 90%|█████████ | 9/10 [00:19<00:02,  2.16s/it]

Iteración 10 de 10


100%|██████████| 10/10 [00:21<00:00,  2.18s/it]


In [162]:
ratings = np.asarray(ratings)
valids = ~np.isnan(ratings_np)
valids3D = valids[..., np.newaxis]
full_shape = *ratings_np.shape, NUM_FACTORS
for it in tqdm(range(NUM_ITERATIONS)):
    print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))
    
    predictions = p_np @ q_np.T
    errors = ratings_np - predictions
    p_np += LEARNING_RATE * (np.broadcast_to(errors[..., np.newaxis], full_shape) * q_np - REGULARIZATION * p_np[:, np.newaxis, :]).sum(axis=1, where=valids3D)
    q_np += LEARNING_RATE * (np.broadcast_to(errors[..., np.newaxis], full_shape) * p_np[:, np.newaxis, :] - REGULARIZATION * q_np[np.newaxis, ...]).sum(axis=0, where=valids3D)

  0%|          | 0/10 [00:00<?, ?it/s]

Iteración 1 de 10


 10%|█         | 1/10 [00:00<00:03,  2.62it/s]

Iteración 2 de 10


 20%|██        | 2/10 [00:00<00:03,  2.53it/s]

...

 80%|████████  | 8/10 [00:03<00:00,  2.36it/s]

Iteración 9 de 10


 90%|█████████ | 9/10 [00:03<00:00,  2.37it/s]

Iteración 10 de 10


100%|██████████| 10/10 [00:04<00:00,  2.43it/s]


## Cálculo de las recomendaciones

El cálculo de las recomendaciones, por lo general, simplemente implica seleccionar los *N* items con una predicción más alta. Por ejemplo, si quisiéramos recomendar *N = 3* items a un usuario que tuviera las siguientes predicciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|   	|  2,9 	|    	|  4,7 	|  5,0 	|    	|  1,2 	|    	|   	|  3,1 	|

Se le recomendarían a dicho usuario los items *i5*, *i4* e *i10*.

In [92]:
N = 5

In [94]:
def get_recommendations_old(predictions):
    recommendations = [None] * N

    for n in range(N):

        max_value = 0
        item = None

        for i, value in enumerate(predictions):
            if i not in recommendations and value != None and value > max_value:
                max_value = value
                item = i

        recommendations[n] = item

    return recommendations

In [ ]:
def get_recommendations(predictions):
    # Hay que tomar una decisión entre que devuelva recomendaciones nulas o que 
    # devuelva un número indeterminado de recomendaciones, opto por la segunda 
    # por eficiencia
    if len(predictions) < N:
        # Not enough recommendations
        recommendations = np.arange(len(predictions))
        recommendations = recommendations[~np.isnan(predictions[recommendations])]
        return recommendations
    

    predictions = np.where(~np.isnan(predictions), predictions, 0)
    recommendations = np.argpartition(predictions, -N)[-N:] # Más eficiente que ordenar
    return recommendations[predictions[recommendations] != 0]

##Ejercicio: Cálculo de Métricas

Calcular el error medio absoluto (MAE) y la raiz del error medio cuadrático (RMSE) de las predicciones realizadas por el algoritmo PMF, así como la precisión, recall, F1 y nDCG de las recomendaciones.

Para ello, lo primero que debemos hacer es calcular las predicciones para todos los items que haya recibido una votación de test:

In [133]:
predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

# Rellenamos la matriz de predicciones
for u in range(NUM_USERS):
    for i in range(NUM_ITEMS):
        if test_ratings[u][i] != None:
            predictions[u][i] = compute_prediction(p[u], q[i])

In [97]:
theta = 4

In [98]:
def get_user_mae (u, predictions):
  mae = 0
  count = 0

  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      mae += abs(test_ratings[u][i] - predictions[u][i])
      count += 1

  if count > 0:
    return mae / count
  else:
    return None

In [99]:
def get_mae (predictions):
  mae = 0
  count = 0

  for u in range(NUM_USERS):
    user_mae = get_user_mae(u, predictions)

    if user_mae != None:
      mae += user_mae
      count += 1

  if count > 0:
    return mae / count
  else:
    return None

In [100]:
import math

def get_user_rmse (u, predictions):
  mse = 0
  count = 0

  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      mse += (test_ratings[u][i] - predictions[u][i]) * (test_ratings[u][i] - predictions[u][i])
      count += 1

  if count > 0:
    return math.sqrt(mse / count)
  else:
    return None

In [101]:
def get_rmse (predictions):
  rmse = 0
  count = 0

  for u in range(NUM_USERS):
    user_rmse = get_user_rmse(u, predictions)

    if user_rmse != None:
      rmse += user_rmse
      count += 1


  if count > 0:
    return rmse / count
  else:
    return None

In [102]:
theta = 4

In [115]:
def get_user_precision (u, predictions):
  precision = 0
  count = 0
  recommendations = get_recommendations_old(predictions[u])

  for i in recommendations:
    if i != None and test_ratings[u][i] != None:
      precision += 1 if test_ratings[u][i] >= theta else 0
      count += 1

  if count > 0:
    return precision / count
  else:
    return None

In [104]:
def get_precision (predictions):
  precision = 0
  count = 0

  for u in range(NUM_USERS):
    user_precision = get_user_precision(u, predictions)

    if user_precision != None:
      precision += user_precision
      count += 1


  if count > 0:
    return precision / count
  else:
    return None

In [117]:
def get_user_recall (u, predictions):
  recall = 0
  count = 0
  recommendations = get_recommendations_old(predictions[u])

  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      if test_ratings[u][i] >= theta:
        recall += 1 if i in recommendations else 0
        count += 1

  if count > 0:
    return recall / count
  else:
    return None

In [106]:
def get_recall (predictions):
  recall = 0
  count = 0

  for u in range(NUM_USERS):
    user_recall = get_user_recall(u, predictions)

    if user_recall != None:
      recall += user_recall
      count += 1


  if count > 0:
    return recall / count
  else:
    return None

In [107]:
def get_user_f1 (u, predictions):
  precision = get_user_precision(u, predictions)
  recall = get_user_recall(u, predictions)

  if precision == None or recall == None:
    return None
  elif precision == 0 and recall == 0:
    return 0
  else:
    return 2 * precision * recall / (precision + recall)

In [108]:
def get_f1 (predictions):
  f1 = 0
  count = 0

  for u in range(NUM_USERS):
    user_f1 = get_user_f1(u, predictions)

    if user_f1 != None:
      f1 += user_f1
      count += 1


  if count > 0:
    return f1 / count
  else:
    return None

In [109]:
def get_ordered_test_items(u):
  num_items = sum(x is not None for x in test_ratings[u])
  items = [None for _ in range(num_items)]

  for n in range(num_items):

    max_value = 0
    item = None

    for i,value in enumerate(test_ratings[u]):
      if i not in items and value != None and value > max_value:
        max_value = value
        item = i

    items[n] = item

  return items

In [110]:
def get_user_idcg (u):
  items = get_ordered_test_items(u)
  idcg = 0

  for pos, i in enumerate(items):
    idcg += (2 ** test_ratings[u][i] - 1) / math.log(pos+2, 2)

  return idcg

In [111]:
def get_user_dcg (u, recommendations):
  dcg = 0

  for pos, i in enumerate(recommendations):
    if i != None and test_ratings[u][i] != None:
      dcg += (2 ** test_ratings[u][i] - 1) / math.log(pos+2, 2)

  return dcg

In [118]:
def get_user_ndcg (u, predictions):
  recommendations = get_recommendations_old(predictions[u])
  dcg = get_user_dcg(u, recommendations)
  idcg = get_user_idcg(u)
  if idcg == 0:
    return 0
  else:
    return dcg / idcg

In [113]:
def get_ndcg (predictions):
  ndcg = 0
  count = 0

  for u in range(NUM_USERS):
    user_ndcg = get_user_ndcg(u, predictions)

    if user_ndcg != None:
      ndcg += user_ndcg
      count += 1


  if count > 0:
    return ndcg / count
  else:
    return None

In [119]:
mae = get_mae(predictions)
rmse = get_rmse(predictions)
precision = get_precision(predictions)
recall = get_recall(predictions)
f1 = get_f1(predictions)
ndcg = get_ndcg(predictions)
print("MAE = " + str(mae))
print("RMSE = " + str(rmse))
print("Precision = " + str(precision))
print("Recall = " + str(recall))
print("F1 = " + str(f1))
print("nDCG = " + str(ndcg))

MAE = 0.9104854697488575
RMSE = 1.0833857192824856
Precision = 0.7119658119658115
Recall = 0.529894197923319
F1 = 0.5275810367587738
nDCG = 0.1189806366379144


In [163]:
predictions = (p_np @ q_np.T).astype(object)
predictions[np.isnan(test_ratings_np)] = None

In [164]:
mae = get_mae(predictions)
rmse = get_rmse(predictions)
precision = get_precision(predictions)
recall = get_recall(predictions)
f1 = get_f1(predictions)
ndcg = get_ndcg(predictions)
print("MAE = " + str(mae))
print("RMSE = " + str(rmse))
print("Precision = " + str(precision))
print("Recall = " + str(recall))
print("F1 = " + str(f1))
print("nDCG = " + str(ndcg))

MAE = 0.8994380838623752
RMSE = 1.0648006812175972
Precision = 0.7078632478632474
Recall = 0.528925545768728
F1 = 0.5247101037151399
nDCG = 0.1191478225635397


## Añadiendo los bias

El modelo descrito anteriormente mejora significativamente la escalabilidad del filtrado colaborativo y, además, incremente notablemente la calidad de las predicciones y recomendaciones. Sin embargo, dicho modelo no se ajusta a la realidad puesto que no refleja los sesgos que los usuarios tienen cuando realizan votaciones.

Parece evidente pensar que no todos los usuarios tienen la misma interpretación de las votaciones. Por ejemplo, existen usuarios más "generosos" con las votaciones que tienden a asignar siempre valoraciones altas y existen usuarios más "tacaños" con las votaciones que tienden a asignar siempre valoraciones más bajas. Que el primer usuario valore un item con 5 y el segundo usuario valore el mismo item con un 4 no quiere decir que al primero le haya gustado más el item. Cada usuario hace su propia interpretación de lo que significan los votos 4 y 5.

Igualmente, existen determinados items que socialmente tienen que gustar y existen otros items que está "mal visto" que gusten. Por ejemplo, resulta extraño que alguien pueda otorgar la nota mínima a *El Padrino* aunque no le haya gustado. La presión social hace que dicha película sea importante, y eso condiciona nuestro voto sobre la misma. Igualmente, resulta extraño que alguien pueda otorgar la nota máxima a *Sharknado* ya que, socialmente, es considerada una película "mala".

Para reflejar este fenómeno dentro de nuestro modelo de factorización matricial, debemos hacer algunas modificaciones sobre el mismo. Para empezar, cambiaremos cómo se calculan las predicciones:

$$\hat{r}_{u,i} = \mu + b_u + b_i + \vec{p}_u \cdot \vec{q}_i$$

Donde $\mu$ representa la votación media de la base de datos, $b_u$ representa el bias (sesgo) del usuario $u$, $b_i$ representa el bias (sesgo) del item $i$ y $\vec{p}_u \cdot \vec{q}_i$ simboliza la interacción entre el usuario $u$ y el item $i$.

De este modo, la predicción será calculada como la media de la base de datos, +/- un ajuste en función de cómo suele votar el usuario, +/- un ajuste de cómo suele votarse el item, y +/- la interacción entre el usuario y el item.

Debido a este cambio, la función a minimizar es ahora la siguiente:

$$\min_{b_u, b_i,p,q} \sum_{(u,i) \in R} ( r_{u,i} - \mu - b_u - b_i - \vec{p}_u \cdot \vec{q}_i)^2 + \lambda (||\vec{p}_u||^2 + ||\vec{q}_i||^2 + b_u^2 + b_i^2)$$

Y ahora toca calcular la derivada respecto de $b_u$, $b_i$, $\vec{p}_u$ y $\vec{q}_i$ para poder aplicar el descenso por gradiente.

Definimos una nueva función para calcular las predicciones:

In [143]:
def compute_biased_prediction(avg, b_u, b_i, p_u, q_i):
    deviation = 0
    for k in range(NUM_FACTORS):
        deviation += p_u[k] * q_i[k]

    prediction = avg + b_u + b_i + deviation
    return prediction

Calculamos el voto medio:

In [144]:
rating_average = 0
rating_count = 0

for u in range(NUM_USERS):
  for i in range(NUM_ITEMS):
    if ratings[u][i] != None:
      rating_average += ratings[u][i]
      rating_count += 1

rating_average /= rating_count

Reiniciamos las matrices de factores y los vectores de bias con valores aleatorios en el intervalo \[0, 1]:

In [179]:
p = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)]
q = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)]

bu = [random.random() for _ in range(NUM_USERS)]
bi = [random.random() for _ in range(NUM_ITEMS)]

In [180]:
p_np = np.array(p) # np.random.random((NUM_USERS, NUM_FACTORS))
q_np = np.array(q) # np.random.random((NUM_ITEMS, NUM_FACTORS))
bu_np = np.array(bu) # np.random.random(NUM_USERS)
bi_np = np.array(bi) # np.random.random(NUM_ITEMS)

Y volvemos a entrenar nuestro modelo:

In [181]:
for it in tqdm(range(NUM_ITERATIONS)):
    print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))

    updated_p = list(p) # clone p matrix
    updated_q = list(q) # clone q matrix

    updated_bu = list(bu) # clone bu vector
    updated_bi = list(bi) # clone bi vector

    for u in range(NUM_USERS):
        for i in range(NUM_ITEMS):
            if ratings[u][i] != None:

                prediction = compute_biased_prediction(rating_average, bu[u], bi[i], p[u], q[i])
                rating = ratings[u][i]
                error = rating - prediction

                for k in range(NUM_FACTORS):
                    updated_p[u][k] += LEARNING_RATE * (error * q[i][k] - REGULARIZATION * p[u][k])
                    updated_q[i][k] += LEARNING_RATE * (error * p[u][k] - REGULARIZATION * q[i][k])

                updated_bu[u] += LEARNING_RATE * (error - REGULARIZATION * bu[u])
                updated_bi[i] += LEARNING_RATE * (error - REGULARIZATION * bi[i])


    p = updated_p
    q = updated_q

    bu = updated_bu
    bi = updated_bi

  0%|          | 0/10 [00:00<?, ?it/s]

Iteración 1 de 10


 10%|█         | 1/10 [00:02<00:18,  2.00s/it]

Iteración 2 de 10


 20%|██        | 2/10 [00:04<00:16,  2.01s/it]

...

 80%|████████  | 8/10 [00:16<00:04,  2.14s/it]

Iteración 9 de 10


 90%|█████████ | 9/10 [00:18<00:02,  2.15s/it]

Iteración 10 de 10


100%|██████████| 10/10 [00:21<00:00,  2.12s/it]


In [195]:
errors

array([[ 1.30501727, -0.54552793,  0.72193678, ...,         nan,
                nan,         nan],
       [ 0.27583127,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [ 0.53390796,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,  1.68065595,         nan, ...,         nan,
                nan,         nan]])

In [182]:
ratings = np.asarray(ratings)
valids = ~np.isnan(ratings_np)
full_shape = *ratings_np.shape, NUM_FACTORS
for it in tqdm(range(NUM_ITERATIONS)):
    print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))
    
    predictions = rating_average + bu_np[:,np.newaxis] + (p_np @ q_np.T) + bi_np
    errors = ratings_np - predictions
    p_np += LEARNING_RATE * (np.broadcast_to(errors[..., np.newaxis], full_shape) * q_np - REGULARIZATION * p_np[:, np.newaxis, :]).sum(axis=1, where=valids[..., np.newaxis])
    q_np += LEARNING_RATE * (np.broadcast_to(errors[..., np.newaxis], full_shape) * p_np[:, np.newaxis, :] - REGULARIZATION * q_np[np.newaxis, ...]).sum(axis=0, where=valids[..., np.newaxis])
    bu_np += LEARNING_RATE * (errors - REGULARIZATION * bu_np[:,np.newaxis]).sum(axis=1, where=valids)
    bi_np += LEARNING_RATE * (errors - REGULARIZATION * bi_np).sum(axis=0, where=valids)

  0%|          | 0/10 [00:00<?, ?it/s]

Iteración 1 de 10


 10%|█         | 1/10 [00:00<00:03,  2.27it/s]

Iteración 2 de 10


 20%|██        | 2/10 [00:00<00:03,  2.14it/s]

...

 80%|████████  | 8/10 [00:03<00:00,  2.19it/s]

Iteración 9 de 10


 90%|█████████ | 9/10 [00:04<00:00,  2.21it/s]

Iteración 10 de 10


100%|██████████| 10/10 [00:04<00:00,  2.19it/s]


##Ejercicio: Cálculo de Métricas

Calcular el error medio absoluto (MAE) y la raiz del error medio cuadrático (RMSE) de las predicciones realizadas por el algoritmo PMF con bias, así como la precisión, recall, F1 y nDCG de las recomendaciones.

Para ello, lo primero que debemos hacer es calcular las predicciones para todos los items que haya recibido una votación de test:

In [202]:
predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

for u in range(NUM_USERS):
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None:
      predictions[u][i] = compute_biased_prediction(rating_average, bu[u], bi[i], p[u], q[i])

In [184]:
mae = get_mae(predictions)
rmse = get_rmse(predictions)
precision = get_precision(predictions)
recall = get_recall(predictions)
f1 = get_f1(predictions)
ndcg = get_ndcg(predictions)
print("MAE = " + str(mae))
print("RMSE = " + str(rmse))
print("Precision = " + str(precision))
print("Recall = " + str(recall))
print("F1 = " + str(f1))
print("nDCG = " + str(ndcg))

MAE = 0.950195917559441
RMSE = 1.1737241676230359
Precision = 0.6268376068376066
Recall = 0.5101915280841663
F1 = 0.49349485223986633
nDCG = 0.10929076005974227


In [204]:
predictions = (rating_average + bu_np[:,np.newaxis] + (p_np @ q_np.T) + bi_np).astype(object)
predictions[np.isnan(test_ratings_np)] = None

In [206]:
mae = get_mae(predictions)
rmse = get_rmse(predictions)
precision = get_precision(predictions)
recall = get_recall(predictions)
f1 = get_f1(predictions)
ndcg = get_ndcg(predictions)
print("MAE = " + str(mae))
print("RMSE = " + str(rmse))
print("Precision = " + str(precision))
print("Recall = " + str(recall))
print("F1 = " + str(f1))
print("nDCG = " + str(ndcg))

MAE = 0.9524264131613294
RMSE = 1.1749433677271972
Precision = 0.6258119658119654
Recall = 0.5070066751201013
F1 = 0.4912390179763843
nDCG = 0.1092170092234227


## Referencias

Mnih, A., & Salakhutdinov, R. R. (2008). **Probabilistic matrix factorization**. In Advances in neural information processing systems (pp. 1257-1264).

Koren, Y., Bell, R., & Volinsky, C. (2009). **Matrix factorization techniques for recommender systems**. Computer, (8), 30-37.